In [ ]:
%matplotlib inline

import os
import os.path as op
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Day 4. Voxel-Based Morphometry on MRiShare dataset
============================================

This example uses Voxel-Based Morphometry (VBM) to study the relationship
between aging, sex and gray matter density.

The data come from the MRiShare database, which have been processed with 
SPM12 New Segment VBM pipeline inside ABACI to create VBM maps.


VBM analysis of aging
---------------------

We run a standard GLM analysis to study the association between age
and gray matter density from the VBM data. We use only 100 subjects
from the MRiShare dataset to limit the memory usage.

Note that more power would be obtained from using a larger sample of subjects.



In [ ]:
# Authors: Bertrand Thirion, <bertrand.thirion@inria.fr>, July 2018
#          Elvis Dhomatob, <elvis.dohmatob@inria.fr>, Apr. 2014
#          Virgile Fritsch, <virgile.fritsch@inria.fr>, Apr 2014
#          Gael Varoquaux, Apr 2014
# Modified by Ami Tsuchida <atsuch@gmail.com>, July, 2019

Examine MRiShare dataset
------------------

### 1. Independent variables used for the analyses


In [ ]:
dat_dir = '../data/'
sub_info = pd.read_csv(op.join(dat_dir, 'sample_mrishare_subinfo.csv'), index_col=0)
sub_info.head()

In [ ]:
n_subjects = len(sub_info)
n_subjects

In [ ]:
# Check number of subjects and age distribution
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8))
sub_info[sub_info.Sex=='M']['Age'].plot.hist(color='b', ax=ax1)
ax1.legend('M')
sub_info[sub_info.Sex=='F']['Age'].plot.hist(color='m', ax=ax2)
ax2.legend('F')

In [ ]:
grouped = sub_info.groupby('Sex')
grouped.describe()

### 2. GM volume image (modulated image in MNI space, 1x1x1mm)

Jacobian modulated GM image in MNI space based on two-channel (i.e. T1 and FLAIR) tissue-segmentation for the same 100 subjects are in the /data/rw_eleves/Cajal-Morphometry2019/copy/anatomical folder.

In [ ]:
mrishare_img_dir = '/data/rw_eleves/Cajal-Morphometry2019/'
anatomical_dir = op.join(mrishare_img_dir, 'copy', 'anatomical')
templates_dir = op.join(mrishare_img_dir, 'templates')

In [ ]:
# Get the path to GM map for each subject in the same order as sub_info.csv

gray_matter_map_filenames = []

for sub in sub_info.index.values:
    gray_matter_map_file_glob = glob.glob(op.join(anatomical_dir,
                                                  sub,
                                                  'jacobian_modulated_gm_image_MNI_111',
                                                  '*gm_segment.nii.gz'))
    if not gray_matter_map_file_glob:
        print ('Could not find the grey matter map for {}!'.format(sub))
    else:
        gray_matter_map_filenames.append(gray_matter_map_file_glob[0])


In [ ]:
# Check to make sure you have expected number of files
len(gray_matter_map_filenames)

In [ ]:
age = sub_info.Age.values
age

Get a mask image: We placed a brain mask image in the template space in a folder 'templates' under mrishare_dat_dir.

Take a look at this mask using the plotting function from nilearn (https://nilearn.github.io/).


In [ ]:
mask = op.join(templates_dir, 'brainmask_111.nii')
template_brain = op.join(templates_dir, 'SHARE500mc_T1brain.nii')

In [ ]:
from nilearn import plotting

In [ ]:
display = plotting.plot_roi(mask, template_brain)

Analyse data
------------

### 1. Create a design matrix
First create an adequate design matrix with three columns: 'age',
'sex', 'intercept'. For sex, sub_info DF encodes it as 'M' and 'F'. Turn this into a binary variable.

In [ ]:
sex = np.array(sub_info.Sex.values == 'M')
sex

 The intercept is simply a column with all 1.

In [ ]:
intercept = np.ones(n_subjects)

Nistats accepts design matrix in the form of pandas dataframe. So we create a numpy array of sex, age, and intercept, and give the column names.

In [ ]:
my_arr = np.vstack((sex, sub_info.Age.values, intercept))
my_arr.shape

The shape indicates that the number of rows is 3. We simply swap the row and column (i.e. transpose) so that the number of columns is 3 for creation of the new DF.

In [ ]:
design_matrix = pd.DataFrame(my_arr.T,
                            columns=['sex', 'age', 'intercept'])
design_matrix.head()

Plot the design matrix



In [ ]:
from nistats.reporting import plot_design_matrix
ax = plot_design_matrix(design_matrix)
ax.set_title('Second level design matrix', fontsize=12)
ax.set_ylabel('maps')

### 2. Fit the model

Specify and fit the second-level model when loading the data, using SecondLevelModel from nistats.second_level_model. We can specify the level of smoothing to be applied to the data (to improve statistical behavior) and also provide a mask to restrict the analysis inside the mask image.


In [ ]:
from nistats.second_level_model import SecondLevelModel
second_level_model = SecondLevelModel(smoothing_fwhm=4.0, mask=mask)
second_level_model.fit(gray_matter_map_filenames,
                       design_matrix=design_matrix)

Estimate the contrast is very simple. We can just provide the column
name of the design matrix (e.g. 'sex'), or manually specify, as below.



In [ ]:
sex_z_map = second_level_model.compute_contrast(second_level_contrast=[1, 0, 0],
                                                output_type='z_score')

Plot the unthresholded zmap using plotting function from nilearn package.

In [ ]:
display = plotting.plot_stat_map(sex_z_map, title='Raw z map for the effect of sex')

You can also plot the thresholded map at uncorrected p < 0.05.
Nistats provide a tool to threshold your map very easily.

In [ ]:
from nistats.thresholding import map_threshold

In [ ]:
thresholded_map1, threshold1 = map_threshold(
    sex_z_map, level=.05, cluster_threshold=0)
print('The uncorrected threshold is %.3g' % threshold1)

In [ ]:
plotting.plot_stat_map(
    sex_z_map, threshold=threshold1, colorbar=True, display_mode='z',
    cut_coords=display.cut_coords,
    title='Sex effect on grey matter density (uncorr p < .05)')

### Correction for multiple comparisons

The map above is actually too liberal, since you are supposed to correct for the fact
that you are testing your model on many, many voxels at the same time!

You can read more about the problem of multiple comparisons in neuroimaging here:

(under resources)
* Zen_and_Art_of_Miltiple_comparisons.pdf
* Non-parametric_methods_and_RFT.pdf
* Nichols_Thresholding.pdf


Nistats offers the basic thresholding using FDR or Bonferoni. I recommend using either SnPM (https://warwick.ac.uk/fac/sci/statistics/staff/academic-research/nichols/software/snpm), FSL randomise (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Randomise), or more newly developped (and still in development) PALM (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/PALM) for the most strict test of your hypotheses, but to get the rough ideas about hox your effects loom like, the basic FDR correction can be good enough, as all the non-parametric methods can take lots of time and memory.

So let's continue looking at the result using a few other ways of thresholding.

First, we threshold at cluster-forming (uncorrected) p <0.001 with only clusters larger than 10 voxels. 

In [ ]:
thresholded_map2, threshold2 = map_threshold(
    sex_z_map, level=.001,  height_control='fpr', cluster_threshold=5)
print('The uncorrected threshold is %.3g' % threshold2)

In [ ]:
plotting.plot_stat_map(
    thresholded_map2, threshold=threshold2, colorbar=True, display_mode='z',
    cut_coords=display.cut_coords,
    title='Thresholded z map, fpr <.001, clusters > 10 voxels')

Next try using FDR threshold of p=0.05.

In [ ]:
thresholded_map3, threshold3 = map_threshold(
    sex_z_map, level=.05, height_control='fdr')
print('The FDR=.05-corrected threshold is: %.3g' % threshold3)

Then try plotting it.

In [ ]:
plotting.plot_stat_map(
    thresholded_map3, threshold=threshold3, colorbar=True, display_mode='z',
    cut_coords=display.cut_coords)

Finally try using FWER correction with Bonferroni.

In [ ]:
thresholded_map4, threshold4 = map_threshold(
    sex_z_map, level=.05, height_control='bonferroni')
print('The p<.05 Bonferroni-corrected threshold is %.3g' % threshold4)

In [ ]:
plotting.plot_stat_map(
    thresholded_map4, threshold=threshold4, colorbar=True, display_mode='z',
    cut_coords=display.cut_coords)

Notice that in Nistats, all of the above thresholding manipulation is one-sided, meaning that you will only see z score passing the threhold on the positive contrast. So if you'd like to look at corrected z score for Female>Male rather than Male>Female, you need to compute z_map with negative contrast (e.g. [-1, 0, 0] to look at Female>Male map.

In Male>Female contrast we used to test diferent threhoslding approaches, we have big effects, regardless of the type of thresholding we use. This is actually to be expected. Can you see why?


## Exercise 1

In section below, try to get the contrast map for the reverse effect, or the effect of age (positive or negative). See if any contrast map survives for the chosen method of multiple comparisons.

## Exercise 2

In the model tested above, we tested the simplest model that assumed no interaction between Age and Sex.

This is not wrong, as long as you have no reason or interest to believe in the possibility of Age x Sex.

How could you modify the design matrix to test the interaction between Age and Sex?

Here are some useful resources if you are not sure;

* GLM and design matirx: background
stat_modeling.pdf
glm.pdf
(under resources folder)

* Design matrix and mean-centering
http://mumford.fmripower.org/mean_centering/

* Other examples of GLM and design matrices
https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM



## Exercise 3

In the sub_info.csv, there is a column 'Score', representing a cognitive test score.
Can you create a new design matrix and contrast to investigate the effect of cognitive score, 
after controlling for age and sex?

## Exercise 4

We used nistats and nilearn to perform the simple GLM analyses and view the results. However, you can use more conventional software packages (FSL, SPM, AFNI, etc) to perform the same analyses. They differ in how you specify desgin matrices/contrasts, how you can control for the multiple comparisons.

Because VBM is typically performed using SPM VBM pipeline, many use SPM for GLM as well. But you don't have to! You should choose the tools based on the features (eg. options for multiple comparison corrections) you desire.

Try using SPM or FSL to perform the same analysis you tried above.